In [1]:

import os
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['ALGICM_BACKEND'] = 'tensorflow'
from algicm.models.backend import functional as F
import tensorflow as tf
import torch

2023-06-16 14:32:47.066781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 14:32:47.236481: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-16 14:32:47.890641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-16 14:32:47.890736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
inputs_tf= F.convert_to_tensor(np.arange(2*3*4*80*80).reshape(2,3,4,80,80))


2023-06-16 14:30:22.972812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 14:30:29.458990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20848 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:83:00.0, compute capability: 8.6


In [ ]:
a,b,c,d,e = [0,0,0,0,0,0],[1,1,1,1,1,1],[2,2,2,2,2,2],[3,3,3,3,3,3],[4,4,4,4,4,4]
index = F.convert_to_tensor(np.array([a,b,d,e]).T)
range_index = F.arange(0,4)


In [8]:
range_index

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>

In [4]:
slice_index = F.expand_dims(F.reshape(F.repeat(range_index, 6),(4,6)),axis=-1)


In [5]:
slice_index = F.cast(slice_index, dtype=tf.int64)


In [6]:
index2= F.repeat(F.expand_dims(index,0),4, axis=0)


In [7]:
index2

<tf.Tensor: shape=(4, 6, 4), dtype=int64, numpy=
array([[[0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4]],

       [[0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4]],

       [[0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4]],

       [[0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 3, 4]]])>

In [10]:
ind = F.concat([index2[:,:,:2],slice_index,index2[:,:,2:]],axis=-1)


In [11]:
ind

<tf.Tensor: shape=(4, 6, 5), dtype=int64, numpy=
array([[[0, 1, 0, 3, 4],
        [0, 1, 0, 3, 4],
        [0, 1, 0, 3, 4],
        [0, 1, 0, 3, 4],
        [0, 1, 0, 3, 4],
        [0, 1, 0, 3, 4]],

       [[0, 1, 1, 3, 4],
        [0, 1, 1, 3, 4],
        [0, 1, 1, 3, 4],
        [0, 1, 1, 3, 4],
        [0, 1, 1, 3, 4],
        [0, 1, 1, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 3, 3, 4],
        [0, 1, 3, 3, 4],
        [0, 1, 3, 3, 4],
        [0, 1, 3, 3, 4],
        [0, 1, 3, 3, 4],
        [0, 1, 3, 3, 4]]])>

In [12]:
tf.transpose(tf.gather_nd(inputs_tf,indices=ind))



<tf.Tensor: shape=(6, 4), dtype=int64, numpy=
array([[25844, 32244, 38644, 45044],
       [25844, 32244, 38644, 45044],
       [25844, 32244, 38644, 45044],
       [25844, 32244, 38644, 45044],
       [25844, 32244, 38644, 45044],
       [25844, 32244, 38644, 45044]])>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["ALGICM_BACKEND"] = "tensorflow"
import tensorflow as tf 
from algicm.registry.common import MODELS
# from algicm.models.pytorch import *

loss_cls_weight = 0.5
loss_bbox_weight = 0.05
loss_obj_weight = 1.0

# The scaling factor that controls the depth of the network structure
deepen_factor = 0.33
# The scaling factor that controls the width of the network structure
widen_factor = 0.5
num_classes = 80
num_det_layers = 3
img_scale = (640, 640)
norm_cfg = dict(type="BN2d", momentum=0.03, eps=0.001)
strides = [8, 16, 32]
anchors = [
    [(10, 13), (16, 30), (33, 23)],  # P3/8
    [(30, 61), (62, 45), (59, 119)],  # P4/16
    [(116, 90), (156, 198), (373, 326)],  # P5/32
]

model_test_cfg = dict(
    # The config of multi-label for multi-class prediction.
    multi_label=True,
    # The number of boxes before NMS
    nms_pre=30000,
    score_thr=0.001,  # Threshold to filter out boxes.
    nms=dict(type="nms", iou_threshold=0.65),  # NMS type and threshold
    max_per_img=300,
)  # Max number of detections of each image


model_cfg = dict(
    type="YOLODetector",
    data_preprocessor=dict(type="BaseDataProcessor"),
    
    backbone=dict(
        type="YOLOv5CSPDarknet",
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        norm_cfg=norm_cfg,
        act_cfg=dict(type="SiLU"),
    ),
    neck=dict(
        type="YOLOv5PAFPN",
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        in_channels=[256, 512, 1024],
        out_channels=[256, 512, 1024],
        num_csp_blocks=3,
        norm_cfg=norm_cfg,
        act_cfg=dict(type="SiLU"),
    ),
    head=dict(
        type="YOLOv5Head",
        head_module=dict(
            type="YOLOv5HeadModule",
            num_classes=num_classes,
            in_channels=[256, 512, 1024],
            widen_factor=widen_factor,
            featmap_strides=strides,
            num_base_priors=3,
        ),
        prior_generator=dict(
            type="YOLOAnchorGenerator", base_sizes=anchors, strides=strides
        ),
        # scaled based on number of detection layers
        loss_cls=dict(
            type="CrossEntropyLoss",
            use_sigmoid=True,
            reduction="mean",
            loss_weight=loss_cls_weight * (num_classes / 80 * 3 / num_det_layers),
        ),
        loss_bbox=dict(
            type="IoULoss",
            iou_mode="ciou",
            bbox_format="xywh",
            eps=1e-7,
            reduction="mean",
            loss_weight=loss_bbox_weight * (3 / num_det_layers),
            return_iou=True,
        ),
        loss_obj=dict(
            type="CrossEntropyLoss",
            use_sigmoid=True,
            reduction="mean",
            loss_weight=loss_obj_weight
            * ((img_scale[0] / 640) ** 2 * 3 / num_det_layers),
        ),
        prior_match_thr=4.0,
        obj_level_weights=[4.0, 1.0, 0.4],
    ),
    test_cfg=model_test_cfg,
)


model = MODELS.build(model_cfg)

2023-07-19 11:47:50.581291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 11:47:50.769413: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-19 11:47:51.675938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-19 11:47:51.676049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
from collections import OrderedDict
def torch2tf(torch_weight_path,tf_model):
 
    '''
    description: Convert torch weights to tf

    param {*} torch_weight_path

    param {*} tf_weight_path

    return {*}
    '''
    all_weights = tf_model.all_weights
    state_dict = {i.name:i.numpy() for i in all_weights}
    state_dict_name = list(state_dict.keys())
    torch_weights = torch.load(torch_weight_path)
    if isinstance(torch_weights,OrderedDict):
        
        for weight in model.weights:
            if weight.name in state_dict_name:
                weight.assign(state_dict[weight.name])
                state_dict.pop(weight.name)
        

        return 1
    elif isinstance(torch_weights,dict):
        assert "state_dict" in torch_weights.keys(),"The key of state_dict should in torch_weights keys"
        torch_weights["state_dict"]
        return



In [4]:
torch2tf("/data/sdv1/zhoutianqi/icm_tlx/icm/pretrain.pth",model)

AttributeError: 'str' object has no attribute 'name'

In [2]:
print(1)
print(1)
print(1)
print(1)
print(1)

1
1
1
1
1


In [1]:
import torch

In [5]:
a = [1,2,3]
if isinstance(a,[list,dict,tuple]):
            print(1)

TypeError: isinstance() arg 2 must be a type or tuple of types